#### Задача 1 (3 балла). 

Разовьем тему с бойцами. Напишите игру (можно взять свой старый код в качестве базы), где игроку будет предложено выбрать класс героя: волшебник или боец. Вы должны учесть возможность добавления новых игровых классов (используйте наследование). У волшебника и бойца немного разные атрибуты (можно атрибуты сделать одинаковые в классе-родителе, но разные коэффициенты в классах-детях, на которые они домножаются: например, здоровье волшебника будет 1.0 от стандартного значения, а здоровье бойца - 1.5, а с маной наоборот). Также у них будут разные методы "нанести удар" и, если хотите, приветствия. Также у нашего героя, кем бы он ни был, должен быть рюкзак, в котором можно рыться и хранить ограниченный набор вещей (в частности, там хранятся зелья: по умолчанию пусть в начале игры каждому персонажу дается по три зелья). Наконец, нужен класс для противника: можете придумать любого монстра (тоже с возможностью добавления новых монстров, очевидно), с которым герой будет сражаться. Во время сражения неплохо предоставлять игроку выбор вида "нанести удар - выпить зелье", а сам урон от удара можно немного рандомизировать с помощью одноименного модуля. Можно еще реализовать и метод sleep для мирного времени, но во время боя он, конечно, не понадобится. 

In [77]:

from random import randint
from random import choice
from time import sleep

class Being:
    def __init__(self):
        self.level = 1
        self.health = 50 * int(self.level)
        

        

class Hero(Being):
    def __init__(self, name):
        super().__init__()
        self.name = name
        self.mana = 50 * int(self.level)
        self.bag = Bag()
        self.super_attack_used = False
        self.battles = 0

    def level_up(self):
        self.level += 1 
        print(f"Уровень {self.name} повышен до {self.level}! Здоровье: {self.health}, мана: {self.mana}")


    def attack(self, monster, super_attack=False):
        '''Герой атакует во время боя, при этом можно выбрать супер-атаку, она забирает больше маны, но наносит больше урона'''
        if super_attack and not self.super_attack_used:
            if self.mana >= 30:
                damage = randint(30, 50)
                self.mana -= 30
                self.super_attack_used = True
                print(f"{self.name} наносит супер-удар! Урон: {damage}, мана: {self.mana}")
            else:
                print(f"Недостаточно маны для супер-удара:()")
                damage = randint(10, 20)
                print(f"{self.name} наносит обычный удар. Урон: {damage}")
        else:
            damage = randint(10, 20)
            print(f"{self.name} наносит удар. Урон: {damage}")

        monster.health -= damage
        return damage
    
    def use_potion(self, potion_name, monster=None):
        '''Применение зелья во время боя'''
        potion = self.bag.find_by_name(potion_name)
        if potion:
            if potion.type == "heal":
                self.health += randint(10, 20)
                print(f"{self.name} выпил {potion.name}. Здоровье: {self.health}")
            elif potion.type == "strength":
                extra_damage = randint(20, 40)
                print(f"{self.name} выпил {potion.name} и наносит усиленный удар. Урон: {extra_damage}")
                monster.health -= extra_damage
            elif potion.type == "freeze" and monster:
                print(f"{self.name} использует {potion.name}. {monster.name} пропускает ход!")
                return True  # Монстр пропустит ход
            self.bag.stuff.remove(potion) 
        else:
            print(f"У {self.name} нет зелья '{potion_name}' в сумке.")

    def use_weapon(self, weapon_name, monster=None):
        '''Применение оружия во время боя'''
        weapon = self.bag.find_by_name(weapon_name)
        if weapon:
            extra_damage = randint(5, 10) * weapon.strength
            print(f"{self.name} использует {weapon.name}, прибавляя {extra_damage} к следующему удару.")
            monster.health -= extra_damage
        else:
            print(f"У {self.name} нет оружия '{weapon_name}' в сумке.")
        
    
    def adventure_time(self):
        '''По сути просто добавляет новые предметы в сумку'''
        if self.mana < 10:
            print(f"Недостаточно маны для поиска артефактов. Мана: {self.mana}")
        else:
            self.mana -= 10 
            print(f"{self.name} отправляется на поиски артефактов! Мана: {self.mana}")
            sleep(3)
            found_object_type = choice(['potion', 'weapon', 'stuff'])
            if found_object_type == 'potion':
                potion = Potion(f"Зелье {choice(['здоровья', 'силы', 'заморозки'])}", choice(['heal', 'strength', 'freeze']))
                self.bag.add_stuff(potion)
                print(f"{self.name} нашел {potion}.")
            elif found_object_type == 'weapon':
                weapon = Weapon(f"Оружие {choice(['меч', 'серп', 'молот'])}", strength=randint(5, 20))
                self.bag.add_stuff(weapon)
                print(f"{self.name} нашел {weapon}.")
            else:
                stuff = Stuff(f"Предмет {choice(['плащ', 'носок', 'перчатки', 'котенок'])}")
                self.bag.add_stuff(stuff)
                print(f"{self.name} нашел {stuff}.")

    def check_the_bag(self):
        for x in self.bag:
            print(x)
        

    def check_the_hero(self):
        return f"Здоровье:{self.health}\nМана: {self.mana}"
    

class Wizard(Hero):
    def __init__(self, name):
        super().__init__(name)
        self.mana *= 1.7

    def __repr__(self):
        return f"Wizard {self.name}, level {self.level}"
    
    def __str__(self):
        return f"Wizard {self.name}, level {self.level}"
    
    def rest(self, time):
        sleep(time)
        self.health += int(time * 0.7)
        self.mana += time
        print(f"{self.name} восстанавливает силы. Здоровье: {self.health}, мана: {self.mana}")


class Knight(Hero):
    def __init__(self, name):
        super().__init__(name)
        self.health *= 1.7

    def __repr__(self):
        return f"Knight {self.name}, level {self.level}"
    
    def __str__(self):
        return f"Knight {self.name}, level {self.level}"

    def rest(self, time):
        sleep(time)
        self.health += time
        self.mana += int(time * 0.7)
        print(f"{self.name} восстанавливает силы. Здоровье: {self.health}, мана: {self.mana}")

class Monster(Being):
     def __init__(self, hero):
        super().__init__()
        self.level = hero.level
        self.health *= 1.3
        self.attack_power = self.level * 6

     def attack(self, hero):
        damage = randint(self.attack_power - 5, self.attack_power + 5)
        hero.health -= damage
        print(f"Монстр атакует {hero.name}, нанося {damage} урона.")
        return damage

    


class Bag:
    '''Большинство функций взяла из домашки))'''
    def __init__(self):
        self.stuff = []


    def add_stuff(self, *stuff):
        for s in stuff:
            self.stuff.append(s)


    def __iter__(self):
        for i in range(len(self.stuff)):
            yield self.stuff[i]

    def find_by_name(self, name):
        object = next((s for s in self.stuff if s.name == name), None)
        return object
    
    def filter_by_origin(self, klass=None):
         '''Позволяет найти в рюкзаке все предметы, относящиеся к классу Potion (klass='potion'), или Weapon (klass='weapon'), или все остальное, в игру я ее вставить, конечно же, забыла...'''
         if klass == 'potion':
             filtered = [a for a in self.artifacts if isinstance(a, Potion)]
         elif klass == 'weapon':
             filtered = [a for a in self.artifacts if isinstance(a, Weapon)]
         else:
             filtered = [a for a in self.artifacts if not isinstance(a, Weapon) and not isinstance(a, Potion)]
         return filtered
    


class Stuff:
    def __init__(self, name):
        self.name = name
        

    def __repr__(self):
        return f"Object {self.name}"
    
    
    def __str__(self):
        return f"{self.name}"
    

class Potion(Stuff):
    def __init__(self, name, type):
        super().__init__(name)
        self.type = type #heal, strengthen, freeze

    
    def __str__(self):
        return f"Potion '{self.name}', type '{self.type}'"


class Weapon(Stuff):
    def __init__(self, name, strength : int):
        super().__init__(name)
        self.strength = strength

    def __str__(self):
        return f"Weapon '{self.name}', strength '{self.strength}'"


In [73]:
class Game:
    def __init__(self):
        print("Начинается игра!")
        hero_name = input("Введите имя героя: ")
        hero_class = input("Выберите класс героя (knight/wizard): ")
        
        # Создаем героя и инициализируем его с предметами
        if hero_class == "wizard":
            self.hero = Wizard(hero_name)
        elif hero_class == "knight":
            self.hero = Knight(hero_name)
        else:
            print("Неверный класс! Создан герой-рыцарь по умолчанию.")
            self.hero = Knight(hero_name)

        # Добавляем 3 случайных предмета в сумку
        self.full_bag()

        # Начинаем игровой процесс
        self.start_game_loop()

    def full_bag(self):
        '''Добавляет три предмета в сумку, для волшебников и рыцарей разные наборы'''
        if isinstance(self.hero, Wizard):
            potions = [Potion(f"Зелье {choice(['здоровья', 'силы', 'заморозки'])}", choice(['heal', 'strength', 'freeze'])) for x in range(2)]
            weapon = Weapon(f"Оружие {choice(['посох', 'волшебная палочка', 'меч'])}", strength=randint(5, 10))
            self.hero.bag.add_stuff(*potions, weapon)
        else:  
            weapon = Weapon(f"Оружие {choice(['меч', 'лук', 'кинжал'])}", strength=randint(5, 15))
            potions = [Potion(f"Зелье {choice(['здоровья', 'силы', 'заморозки'])}", choice(['heal', 'strength', 'freeze'])) for x in range(2)]
            self.hero.bag.add_stuff(weapon, *potions)

    def start_game_loop(self):
        while True:
            print("\nВыберите действие:\n1. Проверить предметы в сумке\n2. Отдохнуть\n3. Отправиться на поиски артефактов\n4. Проверить ману и здоровье\n5. Начать бой\n6. Закончить игру")
            choice = input("Введите номер действия: ")
            if choice == '1':
                self.hero.check_the_bag()
            elif choice == '2':
                time = int(input('Сколько вы хотите отдохнуть? (введите число)'))
                self.hero.rest(time)
            elif choice == '3':
                self.hero.adventure_time()
            elif choice == '4':
                self.hero.check_the_hero()
            elif choice == '5':
                self.start_battle()
            elif choice == '6':
                break
            else:
                print("Неверный выбор. Пожалуйста, попробуйте снова.")


    def start_battle(self):
        monster = Monster(self.hero)
        print(f"Появился монстр уровня: {monster.level}, здоровье: {monster.health}") #уровень монстра будет соответствовать уровню героя
        monster_skipped_turn = False


        while monster.health > 0 and self.hero.health > 0:
            action = input("Выберите действие: 1. Атаковать 2. Использовать зелье или оружие 3. Супер-удар: ")
            if action == '1':
                self.hero.attack(monster)
            elif action == '2':
                chosen_item = input('Выберите, что Вы хотите использовать: зелье или оружие')
                if chosen_item == 'зелье':
                    potion_name = input("Введите имя зелья для использования: ")
                    self.hero.use_potion(potion_name, monster)
                    skipped = self.hero.use_potion(potion_name, monster)
                    if skipped:
                        monster_skipped_turn = True
                else:
                    weapon_name = input("Введите название оружия для использования: ")
                    self.hero.use_weapon(weapon_name, monster)
            elif action == '3':
                self.hero.attack(monster, super_attack=True)
            else:
                print("Неверный выбор.")

            if monster.health > 0 and not monster_skipped_turn:
                monster.attack(self.hero)
            elif monster_skipped_turn:
                print(f"{monster.name} пропускает ход!")
                monster_skipped_turn = False

        if self.hero.health > 0:
            print(f"{self.hero.name} победил!")
            self.hero.battles += 1  
            self.check_level_up()  # Проверка на повышение уровня
        else:
            print(f"{self.hero.name} проиграл.")

    def check_level_up(self):
        '''Проверка условий повышения уровня'''
        if self.hero.battles >= self.hero.level * 3 and len(self.hero.bag.stuff) >= self.hero.level * 5:
            self.hero.level_up()
            

In [76]:
g = Game()

Начинается игра!
Неверный класс! Создан герой-рыцарь по умолчанию.

Выберите действие:
1. Проверить предметы в сумке
2. Отдохнуть
3. Отправиться на поиски артефактов
4. Проверить ману и здоровье
5. Начать бой
6. Закончить игру
Появился монстр уровня: 1, здоровье: 13.0
п наносит удар. Урон: 10
Монстр атакует п, нанося 8 урона.
У п нет зелья 'зелье здоровья' в сумке.
У п нет зелья 'зелье здоровья' в сумке.
Монстр атакует п, нанося 8 урона.
Недостаточно маны для супер-удара:()
Монстр атакует п, нанося 1 урона.
п проиграл.

Выберите действие:
1. Проверить предметы в сумке
2. Отдохнуть
3. Отправиться на поиски артефактов
4. Проверить ману и здоровье
5. Начать бой
6. Закончить игру
Weapon 'Оружие кинжал', strength '14'
Potion 'Зелье силы', type 'strength'
Potion 'Зелье заморозки', type 'freeze'

Выберите действие:
1. Проверить предметы в сумке
2. Отдохнуть
3. Отправиться на поиски артефактов
4. Проверить ману и здоровье
5. Начать бой
6. Закончить игру


#### Задача 2 (3 балла). 

Вспомним задачу токенизации. Напишите собственный простенький токенизатор (с самим процессом можно не сильно заморачиваться), который будет создавать генератор с объектами класса Token, у которых будет атрибут text и атрибут category (латинское слово, кириллическое слово или пунктуация). Токенизатор должен быть реализован в классе, у которого должна быть (генераторная) функция tokenize(). Вам понадобится отдельный класс для токенов и re.finditer(). 

In [46]:
import re


class Token:
    def __init__(self, text):
        self.text = text
        self.category = self.def_category()

    def __repr__(self):
        return f"Token: '{self.text}', category: '{self.category}'"
    
    def __str__(self):
        return f"Token: '{self.text}', category: '{self.category}'"

    def def_category(self):
        '''Определяет категорию токена, все, что не латиница, кириллица и пунктуация, будет считаться числами'''
        if re.match(r'^[а-яА-ЯёЁ]+$', self.text):
            self.category = 'cyrillic'
        elif re.match(r'^[a-zA-Z]+$', self.text):
            self.category = 'latin'
        elif re.match(r'[^\w\s]', self.text):
            self.category = 'punctuation'
        else:
            self.category = 'number'
        return self.category



class Tokenizer:
    def __init__(self):
        pass

    def tokenize(self, text):
        tokens = re.finditer(r'\w+|[^\w\s]', text)
        for t in tokens:
            token = t.group(0) 
            yield Token(token)
          

In [47]:
t = Tokenizer()

In [48]:
for x in t.tokenize('Karl у Клары 4, а Клара у Карла 3.'):
    print(x)

Token: 'Karl', category: 'latin'
Token: 'у', category: 'cyrillic'
Token: 'Клары', category: 'cyrillic'
Token: '4', category: 'number'
Token: ',', category: 'punctuation'
Token: 'а', category: 'cyrillic'
Token: 'Клара', category: 'cyrillic'
Token: 'у', category: 'cyrillic'
Token: 'Карла', category: 'cyrillic'
Token: '3', category: 'number'
Token: '.', category: 'punctuation'
